# Data pre-processing for phonological process error Binary Classification
**Author:** chhsiao<br>
**Date created:** 2022/09/22<br>
**Last modified:** 2022/09/21<br>

## Introduction
利用治療師的標記結果將資料做二分類處理，並且會將單音資料(syllable)重新命名為0和16兩類(e.g. 0=>某種phonological process, 16=>某種phonological process以外的音檔)<br>

注意：<br>
此程式預設字卡為wordcard_BACKING，需要更改字卡請更改**wordcard_dic**<br>
使用前要更改的部份為治療師統計好的標記結果(.csv)的路徑,要計算的構音錯誤TARGET(e.g. 塞音化=>1),以及收音期間
為什麼要把test的字卡跟0327和0820的字卡分開做是因為標記方式不同，如果test的字卡是塞音，個案就算念對，治療師也是把它標塞音

In [1]:
import pandas as pd
import os
import shutil
import json

## SETUP

In [1]:
TARGET = "塞音化"
WORDCARD = "test"
DATE = "0820_0916"
WORDCARD = "wordcard_BACKING"


In [3]:
STATPATH = '/D/TWCC/work/cgh_2022/jsonmv/stat/test/' + DATE+'/'+WORDCARD+'/compareST_'+DATE+'.csv'
DATASET = "binear_classification"


DATASETPATH = '/D/TWCC/work/cgh_2022/data/test/'+WORDCARD+'/use-three-ST-label-dataset_' + \
    DATE+'/' + DATASET+'/'

SYLLABLEPATH = DATASETPATH + 'syllable/'
WORDPATH = DATASETPATH + 'word/'

DATAPATH = '/D/TWCC/data2022_ori/test/'+DATE+'/'


In [4]:
class_dict = {
    "塞音化": 1,
    "母音化": 2,
    "母音省略": 3,
    "舌前音化": 4,
    "舌根音化": 5,
    "不送氣音化": 6,
    "聲隨韻母": 7,
    "邊音化": 8,
    "齒間音": 9,
    "子音省略": 10,
    "擦音化": 11,
    "介音省略": 12,
    "塞擦音化": 13,
    "複韻母省略": 14,
    "其他": 15,
    "正確": 16,
    "雜訊無法辨識": 17
}
class_name = ["塞音化", "母音化", "母音省略", "舌前音化", "舌根音化", "不送氣音化", "聲隨韻母",
              "邊音化", "齒間音", "子音省略", "擦音化", "介音省略", "塞擦音化", "複韻母省略", "其他", "正確", "雜訊無法辨識"]

In [5]:
BACKINGPATH = DATASETPATH + 'Backing/'
save_Backing = 'class_number_wordcard_BACKING'+DATE+'_Backing'

STOPINGPATH = DATASETPATH + 'Stoping/'
save_Stoping = 'class_number_wordcard_BACKING'+DATE+'_Stoping'



# backing_list0327 = [
#     "wordcard03_01_2.wav",
#     "wordcard03_02_2.wav",
#     "wordcard03_07_2.wav",
#     "wordcard03_08_1.wav"
# ]
# backing_list0820 = [
#     "wordcard03_01_2.wav",
#     "wordcard03_02_2.wav",
#     "wordcard03_07_2.wav",
#     "wordcard03_08_1.wav"
# ]
# stoping_list0327 = [
#     "wordcard03_01_1.wav",
#     "wordcard03_02_1.wav",
#     "wordcard03_04_1.wav",
#     "wordcard03_05_2.wav",
#     "wordcard03_08_2.wav",
#     "wordcard04_01_1.wav"
# ]
# stoping_list0820 = [
#     "wordcard03_01_1.wav",
#     "wordcard03_02_1.wav",
#     "wordcard03_04_1.wav",
#     "wordcard03_05_2.wav",
#     "wordcard03_08_2.wav",
#     "wordcard04_01_1.wav",
#     "wordcard05_05_2.wav",
#     "wordcard05_08_1.wav"
# ]


classBackingChildNumber = {
    "非舌根音化": 0,
    "舌根音化": 0
}
classBackingAdultNumber = {
    "非舌根音化": 0,
    "舌根音化": 0
}

classStopingChildNumber = {
    "非塞音化": 0,
    "塞音化": 0
}
classStopingAdultNumber = {
    "非塞音化": 0,
    "塞音化": 0
}


In [6]:
if(TARGET == "舌根音化"):
    classAdultNumber = classBackingAdultNumber
    classChildNumber = classBackingChildNumber
    ERRORPATH = BACKINGPATH
    SAVEERRORNAME = save_Backing
    if not os.path.exists(ERRORPATH):
        os.makedirs(ERRORPATH)
    # if(WORDCARD == "data_0327word"):
    #     error_list = backing_list0327
    # elif(WORDCARD== "data_0820word"):
    #     error_list = backing_list0820
elif(TARGET == "塞音化"):
    classAdultNumber = classStopingAdultNumber
    classChildNumber = classStopingChildNumber
    ERRORPATH = STOPINGPATH
    SAVEERRORNAME = save_Stoping
    if not os.path.exists(ERRORPATH):
        os.makedirs(ERRORPATH)
    # if(WORDCARD == "data_0327word"):
    #     error_list = stoping_list0327
    # elif(WORDCARD == "data_0820word"):
    #     error_list = stoping_list0820


In [7]:
class_dict[TARGET]


1

In [8]:
from ast import literal_eval
df_stat = pd.read_csv(STATPATH, index_col=0)

for case in df_stat.columns.tolist(): 
    for wordcard in df_stat.index.tolist(): 
        if not df_stat[case][wordcard]: 
            pass
        elif (df_stat[case][wordcard] != df_stat[case][wordcard]): 
            pass 
        else: 
            df_stat[case][wordcard] = literal_eval(df_stat[case][wordcard]) # convert to list

FileNotFoundError: [Errno 2] No such file or directory: '/D/TWCC/work/cgh_2022/jsonmv/stat/data_0327word/0401_0811/wordcard_BACKING/compareST_0401_0811.csv'

In [9]:
df_stat.head(20)

NameError: name 'df_stat' is not defined

In [12]:
def isWord(wordcard):
    if(len(wordcard.split('_')) == 2):  # if the wordcard is WORD
        return True
    return False

def isAdult(case):
    if(case.split('_')[-1] == 'adult'):
        return True
    return False


def classPlusOne(ISADULT, TARGET):
    if(ISADULT):
        classAdultNumber[TARGET] += 1
    else:
        classChildNumber[TARGET] += 1


def copy(src, destination):
    if(os.path.exists(src)):
        tmp = os.path.splitext(destination)[0]
        tmpend = tmp.split('/')
        index = tmp.find(tmpend[-1])
        if not os.path.isdir(destination[:index]):
            os.makedirs(destination[:index])
        # print(src, destination)
        # print(destination[:index])
        shutil.copyfile(src, destination)
    else:
        print(f"{src} not found")

for case in df_stat.columns.tolist():
    for wordcard in df_stat.index.tolist():
        # if the case is adult(2022.05.15.19.58.12_946516734_adult)
        if(isAdult(case)):
            tmpNewPath = ERRORPATH + 'adult/'
            ISADULT = True
        else:
            tmpNewPath = ERRORPATH + 'child/'
            ISADULT = False
        ISWORD = isWord(wordcard)
        if(ISWORD):#二分類不考慮詞的情況
            continue
        if not df_stat[case][wordcard]: # if the value is empty
            pass
        elif (df_stat[case][wordcard] != df_stat[case][wordcard]): # if the value is NaN
            pass
        else:

            case_number = case.split('_')[1]  # 2022.04.01.13.12.28_477493581
            # '/D/TWCC/data2022_ori/data_0327word/0401_0707/2022.06.16.09.33.28_1780799170_test/wordcard05_07_1.wav'
            oldFilePath = DATAPATH+case+'/'+wordcard
            # '/D/TWCC/data2022_ori/data_0327word/0401_0707/2022.06.16.09.33.28_1780799170_test/wordcard05_07_1'
            filePath = os.path.splitext(oldFilePath)[0]
            oldFileName = filePath.split('/')[-1]  # 'wordcard05_07_1'
            if (17 in df_stat[case][wordcard]):
                pass
            # if ('fcdp' in wordcard):
            #     pass
            elif(16 in df_stat[case][wordcard]):
                # if(wordcard in error_list):
                #     newFileName = "0_" + str(case_number) + '_' + oldFileName.split('wordcard')[1] + '.wav'
                #     classPlusOne(ISADULT, TARGET)
                #     tmpNewPath = tmpNewPath + '0/'
                # else:
                newFileName = "16_" + str(case_number) + '_' + oldFileName.split('wordcard')[1] + '.wav'
                classPlusOne(ISADULT, "非"+TARGET)
                tmpNewPath = tmpNewPath + '16/'
                if not os.path.isdir(tmpNewPath):
                    os.makedirs(tmpNewPath)
                newFilePath = os.path.join(tmpNewPath, newFileName)
                # print(df_stat[case][wordcard], newFileName)
                # print(f"{oldFilePath} -> {newFilePath}")
                copy(oldFilePath, newFilePath)
                # shutil.copyfile(oldFilePath, newFilePath)
            elif(class_dict[TARGET] in df_stat[case][wordcard]):
                newFileName = "0_" + str(case_number) + '_' + oldFileName.split('wordcard')[1] + '.wav'
                classPlusOne(ISADULT, TARGET)
                tmpNewPath = tmpNewPath + '0/'
                if not os.path.isdir(tmpNewPath):
                    os.makedirs(tmpNewPath)
                newFilePath = os.path.join(tmpNewPath, newFileName)
                # print(f"{oldFilePath} -> {newFilePath}")
                # print(df_stat[case][wordcard], newFileName)
                copy(oldFilePath, newFilePath)
                # shutil.copyfile(oldFilePath, newFilePath)
            elif(15 not in df_stat[case][wordcard]): #排除其他
                newFileName = "16_" + str(case_number) + '_' + oldFileName.split('wordcard')[1] + '.wav'
                classPlusOne(ISADULT, "非"+TARGET)
                tmpNewPath = tmpNewPath + '16/'
                if not os.path.isdir(tmpNewPath):
                    os.makedirs(tmpNewPath)
                newFilePath = os.path.join(tmpNewPath, newFileName)
                # print(f"{oldFilePath} -> {newFilePath}")
                copy(oldFilePath, newFilePath)
                # shutil.copyfile(oldFilePath, newFilePath)



In [13]:
classChildNumber


{'非塞音化': 0, '塞音化': 0}

In [14]:
classAdultNumber


{'非塞音化': 436, '塞音化': 62}

In [15]:
with open(ERRORPATH+SAVEERRORNAME+'child.json', 'w', encoding='utf-8') as f:
    json.dump(classChildNumber, f, ensure_ascii=False, indent=4)
    print("Saving"+ERRORPATH+SAVEERRORNAME+'child.json')
    f.close()


Saving/D/TWCC/work/cgh_2022/data/test/wordcard_BACKING/use-three-ST-label-dataset_0820_0916/binear_classification/Stoping/class_number_0820_0916_Stopingchild.json


In [16]:
with open(ERRORPATH+SAVEERRORNAME+'adult.json', 'w', encoding='utf-8') as f:
    json.dump(classAdultNumber, f, ensure_ascii=False, indent=4)
    print("Saving"+ERRORPATH+SAVEERRORNAME+'adult.json')
    f.close()


Saving/D/TWCC/work/cgh_2022/data/test/wordcard_BACKING/use-three-ST-label-dataset_0820_0916/binear_classification/Stoping/class_number_0820_0916_Stopingadult.json
